In [1]:
from tkinter import ttk
from ttkthemes import ThemedTk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from keras import backend as K
import threading
import time

def recognize_text(image):
    image = cv2.resize(image, (256, 64))
    image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    image = image/255
    prediction = model.predict(image.reshape(1,256,64,1))
    decoded = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],greedy=True)[0][0])   
    alphabet = 'AMOVEtospr.GaikelfmngyLbuPdwhjc,xFWT-\'vBH"S19583#qNRDKUIC(40)2:JY7;zZ6?*XQ!/&'
    text = ''
    for ch in decoded[0]:
        if ch == -1:
            break
        else:
            text += alphabet[ch]
    return text

def open_image():
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.png")])
    if file_path:
        image = Image.open(file_path)
        image.thumbnail((400, 400))
        progressbar['value']+=20
        if image.mode == 'L':
            cv_image = np.array(image)
        else:
            cv_image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2GRAY)
        
        photo = ImageTk.PhotoImage(image)
        image_label.config(image=photo)
        image_label.photo = photo
        text = recognize_text(cv_image)
        progressbar['value']+=50
        time.sleep(1)
        recognized_text.config(text='Predicted text: '+text)
        progressbar['value']+=10
        progressbar['value']=0
        
def open_image_thread():
    thread = threading.Thread(target=open_image)
    thread.start()

model = load_model('./model2.keras', compile = False)   

root = ThemedTk(theme="yaru")
root.title('Handwritten text recognition')
root.geometry('800x400')

root.columnconfigure(index=0, weight=2)
root.columnconfigure(index=1, weight=1)
root.rowconfigure(index=0, weight=1)
root.rowconfigure(index=1, weight=2)
root.rowconfigure(index=2, weight=1)
root.rowconfigure(index=3, weight=1)


open_button = ttk.Button(root, text='Open Image', command=open_image_thread, padding=8)
open_button.grid(row = 2, column = 0, columnspan = 2)

# Create a label to display the image
image_label = ttk.Label(root)
image_label.grid(row = 0, column = 0, rowspan = 2)

recognized_text = ttk.Label(root)
recognized_text.grid(row = 1, column = 1)

progressbar =  ttk.Progressbar(orient="horizontal", mode="determinate", length = 200)
progressbar.grid(row =3, column = 0, columnspan = 2)

root.mainloop()


1/1 [==============================] - 0s 23ms/step
